In [6]:
import hashlib
import uuid
from datetime import datetime
import time

class Account:
    def __init__(self, name):
        self.name = name
        self.id = str(uuid.uuid4())  
        self.hashed_id = self._hash_id(self.id)
        self.balance = 0.0
        self.transactions = []
        self.virtual_card = None
        self.budget = Budget(self)
        self.rewards = Rewards(self)

    def _hash_id(self, input_id):
        return hashlib.sha256(input_id.encode()).hexdigest()

    def deposit(self, amount, from_account=None):
        self.balance += amount
        detail = {
            'type': 'deposit',
            'amount': amount,
            'from': from_account.hashed_id if from_account else 'external',
            'to': self.hashed_id,
            'time': datetime.now().isoformat()
        }
        self.transactions.append(detail)

    def withdraw(self, amount, to_account):
        if amount > self.balance:
            raise ValueError("Balance not enough")
        self.balance -= amount
        detail = {
            'type': 'withdraw',
            'amount': amount,
            'from': self.hashed_id,
            'to': to_account.hashed_id,
            'time': datetime.now().isoformat()
        }
        self.transactions.append(detail)

    def transfer(self, amount, to_account):
        self.withdraw(amount, to_account)
        to_account.deposit(amount, from_account=self)

    def get_balance(self):
        return self.balance

    def get_transactions(self):
        return self.transactions

    def apply_interest(self, interest_rate):
        interest = self.balance * (interest_rate / 100)
        self.balance += interest
        detail = {
            'type': 'interest',
            'amount': interest,
            'from': 'system',
            'to': self.hashed_id,
            'time': datetime.now().isoformat()
        }
        self.transactions.append(detail)


class VirtualCard:
    def __init__(self, account):
        self.card_id = str(uuid.uuid4())
        self.card_balance = 0.0
        self.account = account
        account.virtual_card = self

    def load_card(self, amount):
        if self.account.balance >= amount:
            self.account.balance -= amount
            self.card_balance += amount
            print(f"\nCard {self.card_id} loaded with {amount}. New card balance: {self.card_balance}")
        else:
            print("\nInsufficient funds to load the card.")


class Budget:
    def __init__(self, account):
        self.account = account
        self.budgets = {}

    def create_budget(self, category, amount):
        self.budgets[category] = {'limit': amount, 'spent': 0}

    def add_expense(self, category, amount):
        if category in self.budgets and self.budgets[category]['spent'] + amount <= self.budgets[category]['limit']:
            self.budgets[category]['spent'] += amount
            print(f"\nAdded {amount} to {category} budget. Remaining: {self.budgets[category]['limit'] - self.budgets[category]['spent']}")
        else:
            print(f"\nBudget exceeded for category {category}!")


class Rewards:
    def __init__(self, account):
        self.account = account
        self.points = 0

    def add_points(self, amount):
        earned_points = int(amount / 10)  
        self.points += earned_points
        print(f"\n{earned_points} points added to your account. Total points: {self.points}")

    def redeem_points(self, points_to_redeem):
        if self.points >= points_to_redeem:
            self.points -= points_to_redeem
            print(f"\nRedeemed {points_to_redeem} points. Remaining points: {self.points}")
        else:
            print("\nNot enough points to redeem.")


class AutoTransaction:
    def __init__(self, account, target_account, amount, interval_seconds):
        self.account = account
        self.target_account = target_account
        self.amount = amount
        self.interval_seconds = interval_seconds

    def start(self):
        while True:
            time.sleep(self.interval_seconds)
            self.account.withdraw(self.amount, self.target_account)
            print(f"\nAutomatically withdrew {self.amount} to {self.target_account.name}")


class Notifications:
    def __init__(self, account):
        self.account = account

    def send_notification(self, message):
        print(f"\nNotification for {self.account.name}: {message}")


if __name__ == "__main__":
    person_one = Account("Person One")
    person_two = Account("Person Two")

    person_one.deposit(1000)  
    person_one.apply_interest(5)  

    person_one.transfer(200, person_two)  

    card = VirtualCard(person_one)
    card.load_card(500)  

    person_one.budget.create_budget("Travel", 1000)
    person_one.budget.add_expense("Travel", 200)

    person_one.rewards.add_points(100)

    notification_system = Notifications(person_one)
    notification_system.send_notification("Your deposit of 200 was successful.")

    print("\n==== Person One Account Details ====")
    print(f"Name: {person_one.name}")
    print(f"Balance: {person_one.get_balance()}")
    print("Recent Transactions:")
    for t in person_one.get_transactions():
        print(f"- Type: {t['type']}, Amount: {t['amount']}, Time: {t['time']}")
    
    print("\n==== Person Two Account Details ====")
    print(f"Name: {person_two.name}")
    print(f"Balance: {person_two.get_balance()}")
    print("Recent Transactions:")
    for t in person_two.get_transactions():
        print(f"- Type: {t['type']}, Amount: {t['amount']}, Time: {t['time']}")
    
    print("\n==== Rewards Summary ====")
    print(f"Total Points for {person_one.name}: {person_one.rewards.points}")
    
    print("\n==== Budget Summary ====")
    print("Travel Budget:")
    print(f"Limit: 1000, Spent: {person_one.budget.budgets['Travel']['spent']}")



Card 0dc25379-e1e0-4bec-a17b-0dd311252ef7 loaded with 500. New card balance: 500.0

Added 200 to Travel budget. Remaining: 800

10 points added to your account. Total points: 10

Notification for Person One: Your deposit of 200 was successful.

==== Person One Account Details ====
Name: Person One
Balance: 350.0
Recent Transactions:
- Type: deposit, Amount: 1000, Time: 2025-05-06T20:43:24.085427
- Type: interest, Amount: 50.0, Time: 2025-05-06T20:43:24.085453
- Type: withdraw, Amount: 200, Time: 2025-05-06T20:43:24.085465

==== Person Two Account Details ====
Name: Person Two
Balance: 200.0
Recent Transactions:
- Type: deposit, Amount: 200, Time: 2025-05-06T20:43:24.085475

==== Rewards Summary ====
Total Points for Person One: 10

==== Budget Summary ====
Travel Budget:
Limit: 1000, Spent: 200
